# Differential Methylation w/ Statistical Models

## Setup

In [26]:
import pandas as pd
import pysam
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from intervaltree import Interval, IntervalTree

### Helper Functions

In [27]:
def read_bedmethyl(bed_path):
    # Load the bedMethyl file
    df = pd.read_csv(bed_path, sep='\t', header=None)
    # Name the columns based on the bedMethyl format
    df.columns = ['chrom', 'start', 'end', 'name', 'score', 'sample_a_counts', 'sample_a_total', 'sample_b_counts', 'sample_b_total', 'sample_a_fractions', 'sample_b_fractions']
    # Filter rows where 'score' is not zero
    df_filt = df[df['score'] != 0]
    # Sort by 'score' in descending order
    df_filt = df_filt.sort_values('score', ascending=False)
    return df_filt

In [28]:
def find_overlapping_regions(dmr_df, gene_df):
    # Create IntervalTree from gene intervals
    gene_tree = IntervalTree()
    for row in gene_df.itertuples(index=False):
        gene_tree[row.start:row.end] = row.gene_symbol  # Assuming 'gene' is the column with gene names

    # Find overlapping intervals
    overlapping_intervals = []
    overlapping_genes = []
    for row in tqdm(dmr_df.itertuples(index=False), total=len(dmr_df)):
        overlapping_gene = gene_tree[row.start:row.end]
        if overlapping_gene:
            overlapping_intervals.append(row)
            overlapping_genes.append(overlapping_gene.pop().data)  # Get the gene name from the Interval object

    # Create DataFrame from overlapping intervals
    overlapping_df = pd.DataFrame(overlapping_intervals, columns=dmr_df.columns)
    overlapping_df['gene_symbol'] = overlapping_genes  # Add the overlapping genes as a new column

    return overlapping_df

### Data Import

#### Gene Bodies

In [23]:
gene_bodies_105 = pd.read_csv("/data/lvisser/segmentations/gene_bodies_105.csv", dtype={2: str})
print(gene_bodies_105.shape)
gene_bodies_105.head()

(39620, 6)


,start,end,chrom,gene_symbol,segment_id,length
0,577,647,chrMT,MT-TF,chrMT:577-647,70
1,648,1601,chrMT,MT-RNR1,chrMT:648-1601,953
2,1602,1670,chrMT,MT-TV,chrMT:1602-1670,68
3,1671,3229,chrMT,MT-RNR2,chrMT:1671-3229,1558
4,3230,3304,chrMT,MT-TL1,chrMT:3230-3304,74


## Haplotypes

### CGIs

In [24]:
imr_phased_dmr = read_bedmethyl('/data/lvisser/modkit/outputs/bedmethyl/nb1_haplo/IMR14nov_R1.haplotagged.sorted/IMR14nov_R1.haplotagged.sorted_1_IMR14nov_R1.haplotagged.sorted_2_dmr_cgi_result.bed')
print(imr_phased_dmr.shape)
imr_phased_dmr.head()

(5720, 11)


,chrom,start,end,name,score,sample_a_counts,sample_a_total,sample_b_counts,sample_b_total,sample_a_fractions,sample_b_fractions
15382,chr20,33667005,33668183,CpG: 111,1512.274284,"h:8,m:22",935,"h:52,m:1586",1644,"h:0.86,m:2.35","h:3.16,m:96.47"
15961,chr21,8431968,8441142,CpG: 1046,803.982130,"h:6800,m:42677",82096,"h:1406,m:8604",22160,"h:8.28,m:51.98","h:6.34,m:38.83"
4735,chr12,57726126,57727768,CpG: 150,567.634190,"h:38,m:1050",2187,"h:39,m:1188",1241,"h:1.74,m:48.01","h:3.14,m:95.73"
359,chr1,65002589,65003145,CpG: 59,315.837050,"h:16,m:427",485,"h:18,m:300",1174,"h:3.30,m:88.04","h:1.53,m:25.55"
21422,chr6,144007779,144008710,CpG: 111,295.984654,"h:3,m:6",303,"h:10,m:213",230,"h:0.99,m:1.98","h:4.35,m:92.61"


In [29]:
overlapping_df = find_overlapping_regions(imr_phased_dmr, gene_bodies_105)
print(overlapping_df.shape)
overlapping_df.head()

100%|██████████| 5720/5720 [00:00<00:00, 9026.63it/s]

(5701, 12)


,chrom,start,end,name,score,sample_a_counts,sample_a_total,sample_b_counts,sample_b_total,sample_a_fractions,sample_b_fractions,gene_symbol
0,chr20,33667005,33668183,CpG: 111,1512.274284,"h:8,m:22",935,"h:52,m:1586",1644,"h:0.86,m:2.35","h:3.16,m:96.47",CHST8
1,chr21,8431968,8441142,CpG: 1046,803.982130,"h:6800,m:42677",82096,"h:1406,m:8604",22160,"h:8.28,m:51.98","h:6.34,m:38.83",NXPH1
2,chr12,57726126,57727768,CpG: 150,567.634190,"h:38,m:1050",2187,"h:39,m:1188",1241,"h:1.74,m:48.01","h:3.14,m:95.73",ATP8B1
3,chr1,65002589,65003145,CpG: 59,315.837050,"h:16,m:427",485,"h:18,m:300",1174,"h:3.30,m:88.04","h:1.53,m:25.55",SLC1A4
4,chr6,144007779,144008710,CpG: 111,295.984654,"h:3,m:6",303,"h:10,m:213",230,"h:0.99,m:1.98","h:4.35,m:92.61",PLAGL1
